<a href="https://colab.research.google.com/github/Rogerio-mack/IA_2024S2/blob/main/IA_P2_2024S2_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

In [1]:
#@markdown imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif

from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



# Case: Blood transfusion

Considere os dados abaixo sobre indivíduos potenciais doadores de sangue.

In [2]:
df = pd.read_csv("https://github.com/Rogerio-mack/IA_2024S2/raw/refs/heads/main/data/blood_donate.csv")
df.head()

,Recency,Frequency,Monetary,Time,Class
0,2,50,12500,98,donated
1,0,13,3250,28,donated
2,1,16,4000,35,donated
3,2,20,5000,45,donated
4,1,24,6000,77,not donated


# Ex1. Mutual information (Ganho de Informação)

Selecione os 3 atributos que mais apresentam ganho de informação para determinar a classe do indivíduo (doador ou não).

**Nota:**

1. Empregue `random_state=42` em sua função.

In [3]:
from sklearn.feature_selection import mutual_info_classif

X = df.drop(['Class'], axis=1)
y = df['Class']

mutual_info = mutual_info_classif(X, y, random_state=42)
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Mutual Information': mutual_info})
feature_importance_df = feature_importance_df.sort_values('Mutual Information', ascending=False)

print(feature_importance_df)

     Feature  Mutual Information
0    Recency            0.066364
2   Monetary            0.032154
1  Frequency            0.021471
3       Time            0.000000


### **Q1. Quais os 3 atributos que mais apresentam ganho de informação para determinar a classe do indivíduo (doador ou não)?**

In [4]:
Q1 = feature_importance_df.head(3)

# Ex2. Classification

Empregando **somente os 3 atributos preditores com maior ganho de informação**, avalie como `GridSearchCV()` os seguintes estimadores para Classe dos indivíduos:

<br>

> **Estimadores**

* `DecisionTreeClassifier()`, variando a profundidade da árvore de 5-10 e os critérios de ganho de informação 'gini','entropy','log_loss'.

* `MLPClassifier(max_iter=500)`, variando o número de neurônios da camada oculta para uma única camada com 100 elementos, e duas camadas com (10,50), (50,10) elementos respectivamente, além das funções de ativação logística e `relu`.

> Em ambos os casos não esqueça de empregar nos estimadores `random_state=42` para a garantia da reprodutibilidade dos resultados. Você pode optar por empregar um laço de programa para avaliar os dois estimadores ou fazer avaliações separadas. Fica a seu critério.

<br>


> **Conjunto de Treinamento e Teste, CV**

* Empregue 25% de dados de teste, **não estratificados**. Não esqueça de empregar `random_state=42` para a garantia da reprodutibilidade dos resultados. Empregue 5 partições de CV.

> Para a garantia da reprodutibilidade dos resultados **não empregue outros parâmetros que não foram solicitados.**

<br>


> **Pré-Processamento dos dados**

* Aplique a normalização `StandardScaler()`.


In [5]:
#
# Separa os conjuntos de Treinamento e Teste
#
from sklearn.model_selection import train_test_split

X = df.drop(['Class','Time'], axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((561, 3), (561,), (187, 3), (187,))

In [6]:
#
# Normaliza os dados
#
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
#@markdown sanity check, must be True
X_test_scaled.sum().sum() == -65.76431886866018

True

In [8]:
#
# Avalia com o GridSearchCV() os diferentes estimadores e seus parâmetros
#
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Q21 = []

base_estimators = {
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'MLPClassifier': MLPClassifier(random_state=42,max_iter=500)
}

parm_grid = {
    'DecisionTreeClassifier': {'max_depth': range(5, 11),'criterion': ['gini','entropy','log_loss']},
    'MLPClassifier': {'hidden_layer_sizes': [(100,), (10,50), (50,10)], 'activation': ['relu', 'logistic']}
}

for name, base_estimator in base_estimators.items():
    print(f"Training {name}...")
    clf = GridSearchCV(base_estimator, parm_grid[name], cv=5)
    clf.fit(X_train, y_train)

    print(f'best estimator: {clf.best_estimator_} with score {clf.best_score_:.3f}')
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    print(f'accuracy score in X_test: {clf.score(X_test, y_test):.3f}')

    Q21.append((name, clf.best_estimator_, clf.score(X_test, y_test)))


Training DecisionTreeClassifier...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


best estimator: DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42) with score 0.768
              precision    recall  f1-score   support

     donated       0.37      0.23      0.28        48
 not donated       0.76      0.86      0.81       139

    accuracy                           0.70       187
   macro avg       0.57      0.55      0.55       187
weighted avg       0.66      0.70      0.68       187

accuracy score in X_test: 0.701
Training MLPClassifier...
best estimator: MLPClassifier(hidden_layer_sizes=(50, 10), max_iter=500, random_state=42) with score 0.770
              precision    recall  f1-score   support

     donated       0.00      0.00      0.00        48
 not donated       0.74      1.00      0.85       139

    accuracy                           0.74       187
   macro avg       0.37      0.50      0.43       187
weighted avg       0.55      0.74      0.63       187

accuracy score in X_test: 0.743


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **Q2.1.A. Qual o melhor modelo de Árvore de Decisão obtido e sua acuracidade no conjunto de Teste**?

## **Q2.1.B. Qual o melhor modelo de Multilayer Perceptron obtido e sua acuracidade no conjunto de Teste**?


## **Q2.2. Ao final qual o melhor estimador a ser empregado?**

Atenção: Avalie o `Classification Report`.

In [9]:
Q22 = 'DecisionTreeClassifier'

# Ex3. PCA

Verifique o resultado do melhor modelo do exercício anterior (modelo appontado na quesão 3) com o uso de Componentes Principais que correspondam a 80% da variância dos dados.

<br>

> **Reconstrua os conjuntos de Treinamento e Teste como abaixo**

* Todas as features serão empregadas e os dados não precisam ser normalizados.

<br>

> **Em seguida aplique o o PCA aos dados de Treinamento e Teste**

<br>

> **Aplique o estimador de classificação (Q3.) aos novos dados redimensionados**


In [10]:
#
# Separa os conjuntos de Treinamento e Teste
#
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((561, 4), (561,), (187, 4), (187,))

In [11]:
#
# Aplica o PCA
#
from sklearn.decomposition import PCA

pca = PCA(n_components=0.8)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print(f"Número de componentes principais retidos: {pca.n_components_} com {pca.explained_variance_ratio_} da variância dos dados.")


Número de componentes principais retidos: 1 com [0.99981604] da variância dos dados.


In [12]:
#@markdown sanity check, must be True
X_test_pca.sum().sum() == -37083.466947960005

False

In [13]:
#
# Aplica o estimador de classificação (Q3.) aos novos dados redimensionados
#
clf = DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)

print(classification_report(y_test, y_pred))
print(f'accuracy score in X_test: {clf.score(X_test_pca, y_test):.3f}')



              precision    recall  f1-score   support

     donated       0.80      0.08      0.15        48
 not donated       0.76      0.99      0.86       139

    accuracy                           0.76       187
   macro avg       0.78      0.54      0.51       187
weighted avg       0.77      0.76      0.68       187

accuracy score in X_test: 0.759


## **Q3.1. Qual a quantidade de componentes principais e a variância acumulada por esses componentes?**



In [14]:
Q31 = (pca.n_components_, pca.explained_variance_ratio_)

## **Q3.2. Qual a nova acuracidade obtida no conjunto de Teste?**

In [15]:
Q32 = clf.score(X_test_pca, y_test)

# Ex4. Clustering

Aplique a Clusterização Kmeans e Hierárquica aos dados preditores (isto é, exceto o atributo Class) aplicando um PCA , para formação de 2 grupos de indivíduos. Note, **o pca agora é aplicado a todos os dados, `X`**, e não somente aos dados de treinamento. Portanto, não aplique o estimador anterior!

Responda as questões sobre os grupos, e em seguida verifique a **Acurácia de Clusterização** (isto é, a coincidência com os grupos formados e suas classes).

In [16]:
#
# Aplique o PCA aos dados
#

pca = PCA(n_components=0.8)
X_pca = pca.fit_transform(X)


In [17]:
#@markdown sanity check, must be True
X_pca.sum().sum() == -1.5279510989785194e-10

False

In [18]:
#
# Faça as Clusterizações Kmeans e Hierárquica
#
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_pca)
print(f'Kmeans silhouette_score average: {silhouette_score(X_pca, kmeans.labels_):.3f}, com {sum(kmeans.labels_==0)} e {sum(kmeans.labels_==1)} elementos em cada grupo')

hclust = AgglomerativeClustering(n_clusters=2,linkage='complete')
hclust.fit(X_pca)
print(f'Hclust silhouette_score average: {silhouette_score(X_pca, hclust.labels_):.3f}, com {sum(hclust.labels_==0)} e {sum(hclust.labels_==1)} elementos em cada grupo')




Kmeans silhouette_score average: 0.704, com 117 e 631 elementos em cada grupo
Hclust silhouette_score average: 0.857, com 740 e 8 elementos em cada grupo


## Q4.1.A. Qual a silhueta média obtida na clusterização Hierárquica e o número de indivíduos de cada grupo?

## Q4.1.B. Qual a silhueta média obtida na clusterização Kmeans e o número de indivíduos de cada grupo?

In [19]:
Q41 = {'A':(silhouette_score(X_pca, hclust.labels_),sum(hclust.labels_==0),sum(hclust.labels_==1)),
       'B':(silhouette_score(X_pca, kmeans.labels_),sum(kmeans.labels_==0),sum(kmeans.labels_==1))}

## Q4.2.A. Qual cluster hierárquico apresenta maior renda (`Monetary`) e qual esse valor médio?

## Q4.2.B. Qual cluster kmeans estão associados os indivíduos 0, 5, 6 e 8?

In [20]:
df['hclust'] = hclust.labels_
df['kmeans'] = kmeans.labels_

In [21]:
print(df['hclust'].value_counts())
print(df['kmeans'].value_counts())
print(df['Class'].value_counts())

hclust
0    740
1      8
Name: count, dtype: int64
kmeans
1    631
0    117
Name: count, dtype: int64
Class
not donated    570
donated        178
Name: count, dtype: int64


In [22]:
display(df.drop(columns=['Class','hclust']).groupby('kmeans').mean())
display(df.drop(columns=['Class','kmeans']).groupby('hclust').mean())
display(df.drop(columns=['kmeans','hclust']).groupby('Class').mean())

,Recency,Frequency,Monetary,Time
kmeans,,,,
0,6.666667,15.786325,3946.581197,62.769231
1,10.033281,3.610143,902.535658,29.000000


,Recency,Frequency,Monetary,Time
hclust,,,,
0,9.552703,5.12973,1282.432432,33.637838
1,5.250000,41.12500,10281.250000,93.875000


,Recency,Frequency,Monetary,Time
Class,,,,
donated,5.455056,7.797753,1949.438202,32.719101
not donated,10.771930,4.801754,1200.438596,34.770175


In [23]:
Q42 = {'A':(df.drop(columns=['Class','kmeans']).groupby('hclust').Monetary.mean().argmax(),
            df.drop(columns=['Class','kmeans']).groupby('hclust').Monetary.mean()),
       'B':df.iloc[[0,5,6,8],6]}

In [24]:
df.iloc[[0,5,6,8],:]

,Recency,Frequency,Monetary,Time,Class,hclust,kmeans
0,2,50,12500,98,donated,1,0
5,4,4,1000,4,not donated,0,1
6,2,7,1750,14,donated,0,1
8,2,9,2250,22,donated,0,1


## Q4.3.A. Qual a Acurácia de Clusterização para os clusters kmeans com relação a `Class`?

## Q4.3.B. Qual a Acurácia de Clusterização para os clusters hirárquicos com relação a `Class`?

In [25]:
df['kmeans_Class'] = df['kmeans'].map({0:'not donated', 1:'donated'})
df['hclust_Class'] = df['hclust'].map({0:'not donated', 1:'donated'})

In [26]:
confusion_matrix(df.Class, df['kmeans_Class']), confusion_matrix(df.Class, df['hclust_Class'])

(array([[134,  44],
        [497,  73]]),
 array([[  6, 172],
        [  2, 568]]))

In [27]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(df.Class, df['kmeans_Class'])
print(f"Acurácia de Clusterização (kmeans): {accuracy:.3f}")

accuracy = accuracy_score(df.Class, df['hclust_Class'])
print(f"Acurácia de Clusterização (hclust): {accuracy:.3f}")

Acurácia de Clusterização (kmeans): 0.277
Acurácia de Clusterização (hclust): 0.767


In [28]:
Q43 = {'A': ('kmeans', accuracy_score(df.Class, df['kmeans_Class'])),
       'B': ('hclust', accuracy_score(df.Class, df['hclust_Class']))}

# Gabarito

In [29]:
print(f'Q1. Quais os 3 atributos que mais apresentam ganho de informação para determinar a classe do indivíduo (doador ou não)?')
print(Q1)
print()
print(f'Q2.1.A. Qual o melhor modelo de Árvore de Decisão obtido e sua acuracidade no conjunto de Teste?')
print(Q21[0])
print()
print(f'Q2.1.B. Qual o melhor modelo de Multilayer Perceptron obtido e sua acuracidade no conjunto de Teste?')
print(Q21[1])
print()
print(f'Q2.2. Ao final qual o melhor estimador a ser empregado?')
print(Q22)
print()
print(f'Q3.1. Qual a quantidade de componentes principais e a variância acumulada por esses componentes?')
print(Q31)
print()
print(f'Q3.2. Qual a nova acuracidade obtida no conjunto de Teste?')
print(Q32)
print()
print(f'Q4.1.A. Qual a silhueta média obtida na clusterização Hierárquica e o número de indivíduos de cada grupo?')
print(Q41['A'])
print()
print(f'Q4.1.B. Qual a silhueta média obtida na clusterização Kmeans e o número de indivíduos de cada grupo?')
print(Q41['B'])
print()
print(f'Q4.2.A. Qual cluster hirárquico apresenta maior renda (`Monetary`) e qual esse valor médio?')
print(Q42['A'])
print()
print(f'Q4.2.B. Qual cluster kmeans estão associados os indivíduos 0, 5, 6 e 8?')
print(Q42['B'])
print()
print(f'Q4.3.A. Qual a Acurácia de Clusterização para os clusters kmeans com relação a `Class`?')
print(Q43['A'])
print()
print(f'Q4.3.B. Qual a Acurácia de Clusterização para os clusters hirárquicos com relação a `Class`?')
print(Q43['B'])


Q1. Quais os 3 atributos que mais apresentam ganho de informação para determinar a classe do indivíduo (doador ou não)?
     Feature  Mutual Information
0    Recency            0.066364
2   Monetary            0.032154
1  Frequency            0.021471

Q2.1.A. Qual o melhor modelo de Árvore de Decisão obtido e sua acuracidade no conjunto de Teste?
('DecisionTreeClassifier', DecisionTreeClassifier(criterion='entropy', max_depth=6, random_state=42), 0.7005347593582888)

Q2.1.B. Qual o melhor modelo de Multilayer Perceptron obtido e sua acuracidade no conjunto de Teste?
('MLPClassifier', MLPClassifier(hidden_layer_sizes=(50, 10), max_iter=500, random_state=42), 0.7433155080213903)

Q2.2. Ao final qual o melhor estimador a ser empregado?
DecisionTreeClassifier

Q3.1. Qual a quantidade de componentes principais e a variância acumulada por esses componentes?
(1, array([0.99981604]))

Q3.2. Qual a nova acuracidade obtida no conjunto de Teste?
0.7593582887700535

Q4.1.A. Qual a silhueta média 